In [1]:
import tensorrt as trt
import os
import json

In [ ]:
print(f"Starting export with TensorRT {trt.__version__}...")
logger = trt.Logger(trt.Logger.INFO)

# Engine builder
builder = trt.Builder(logger)
config = builder.create_builder_config()

In [5]:
# config.max_workspace_size = 2
flag = 1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)
network = builder.create_network(flag)
half = True

# Read ONNX file
parser = trt.OnnxParser(network, logger)
if not parser.parse_from_file('model.onnx'):
    raise RuntimeError(f"failed to load ONNX file: {f_onnx}")

# Network inputs
inputs = [network.get_input(i) for i in range(network.num_inputs)]
outputs = [network.get_output(i) for i in range(network.num_outputs)]
for inp in inputs:
    print(f'input "{inp.name}" with shape{inp.shape} {inp.dtype}')
for out in outputs:
    print(f'output "{out.name}" with shape{out.shape} {out.dtype}')

[12/05/2024-03:11:18] [TRT] [I] ----------------------------------------------------------------
[12/05/2024-03:11:18] [TRT] [I] Input filename:   model.onnx
[12/05/2024-03:11:18] [TRT] [I] ONNX IR version:  0.0.7
[12/05/2024-03:11:18] [TRT] [I] Opset version:    14
[12/05/2024-03:11:18] [TRT] [I] Producer name:    pytorch
[12/05/2024-03:11:18] [TRT] [I] Producer version: 1.11.0
[12/05/2024-03:11:18] [TRT] [I] Domain:           
[12/05/2024-03:11:18] [TRT] [I] Model version:    0
[12/05/2024-03:11:18] [TRT] [I] Doc string:       
[12/05/2024-03:11:18] [TRT] [I] ----------------------------------------------------------------
input "images" with shape(1, 3, 224, 224) DataType.FLOAT
output "output0" with shape(1, 2) DataType.FLOAT


In [6]:
if half:
    config.set_flag(trt.BuilderFlag.FP16)

In [10]:
# Write file
build = builder.build_engine
with build(network, config) as engine, open('model.engine', "wb") as t:
    # Metadata
    meta = json.dumps({'model': 'mobilenet_v2', 'athor': 'vdvu'})
    t.write(len(meta).to_bytes(4, byteorder="little", signed=True))
    t.write(meta.encode())
    # Model
    t.write(engine.serialize())

/tmp/ipykernel_737/1153334667.py:3: DeprecationWarning: Use build_serialized_network instead.
  with build(network, config) as engine, open('model.engine', "wb") as t:


[12/05/2024-03:13:52] [TRT] [I] ---------- Layers Running on DLA ----------
[12/05/2024-03:13:52] [TRT] [I] ---------- Layers Running on GPU ----------
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_0 + PWN(Clip_3)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_4 + PWN(Clip_7)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_8
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_9 + PWN(Clip_12)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_13 + PWN(Clip_16)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_17
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_18 + PWN(Clip_21)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_22 + PWN(Clip_25)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_26 + Add_27
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_28 + PWN(Clip_31)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_32 + PWN(Clip_35)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_36
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv_37 + PWN(Clip_40)
[12/05/2024-03:13:52] [TRT] [I] [GpuLayer] Conv

In [12]:
!pip3 freeze > requirements.txt